In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import GPy

import time as t

In [2]:
import pandas as pd
import sys
from env_pybullet_gen3 import env_pybullet_kin_gen3



In [3]:
#Create a experiment env
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Initially parameters of the urdf

#Make maxvels closer to the reality
#both have to be modified
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
print(env.original_parameters_df["max_vel"])
print(env.original_parameters_df)


print('observation space:', env.observation_space) #states, There is only 1 state constant
env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous




hola
../Simulation_Pybullet/models/urdf/JACO3_URDF_V11.urdf
Robot launched
hola
(7, 12)
(7, 17)
0    168
1    151
2    128
3     94
4    210
5     48
6    189
Name: max_vel, dtype: int64
       mass damping       Ixx       Iyy       Izz   kp   ki   kd  max_vel  \
0  1.377353       0  0.004801  0.004755  0.002283  0.1  0.0  0.0      168   
1  1.163667       0  0.008419  0.001920  0.008361  0.1  0.0  0.0      151   
2  1.163660       0  0.007545  0.007487  0.001921  0.1  0.0  0.0      128   
3  0.930287       0  0.006410  0.001380  0.006518  0.1  0.0  0.0       94   
4  0.678106       0  0.001680  0.001506  0.000826  0.1  0.0  0.0      210   
5  0.678106       0  0.001938  0.000827  0.001763  0.1  0.0  0.0       48   
6  0.500657       0  0.000775  0.000585  0.000975  0.1  0.0  0.0      189   

   force_x_one  
0            1  
1            1  
2            1  
3            1  
4            1  
5            1  
6            1  
observation space: 1
mass okey
max_vel okey
kp okey
ki okey


In [4]:
#Create a first search
pop_size = 70*20
sigma = 0.3
original_action = np.array(env.action_original())
np.random.seed(0)
Create = False
rewards = []
actions = []

if(Create ==True):
    #Generate new population weights to test
    weights_pop = [(sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
    for weights in weights_pop:
                    action=np.add(np.multiply(weights,original_action),original_action)
                    actions.append(action)
                    rewards.append(env.step_tcp_rishabh(action))
    actions = np.array(actions)
    rewards = np.array(rewards).reshape(actions.shape[0],1)
    
    i_data = np.hstack((actions,rewards))
    np.save("i_data.npy",i_data)
else:
    i_data = np.load("i_data.npy")
    actions = i_data[:,:-1]
    rewards = i_data[:,-1:]


In [5]:
print("actions")
print(actions.shape)
print(actions)
print("rewards")
print(rewards.shape)
print(rewards)

actions
(1400, 63)
[[2.10626984e+00 1.30336192e+00 1.50533547e+00 ... 6.59419705e-04
  1.57288567e-03 7.37282918e-04]
 [6.64042844e-01 1.22560648e+00 1.02339908e+00 ... 1.15477545e-03
  1.39569038e-03 9.31393856e-04]
 [1.19754499e+00 1.80924515e+00 1.39835676e+00 ... 5.52380219e-04
  1.79065083e-03 7.58809772e-04]
 ...
 [1.97407477e+00 6.37037773e-01 1.16377190e+00 ... 1.12702430e-03
  1.26605998e-03 1.19178768e-03]
 [7.96462938e-01 1.29488958e+00 9.74594365e-01 ... 9.36204322e-04
  1.23476868e-03 1.05441314e-03]
 [1.28513665e+00 6.25185039e-01 1.18711875e+00 ... 7.00949285e-04
  2.19622534e-03 1.02368897e-03]]
rewards
(1400, 1)
[[-0.15615385]
 [-0.17645011]
 [-0.32277424]
 ...
 [-0.21721271]
 [-0.19800912]
 [-0.16752433]]


In [6]:
#Create a Gaussian model, which models the relation between parameters and score
def Create_Gaussian_Model (actions,rewards):

    kernel = GPy.kern.Linear(actions.shape[1], ARD = 1)
    #kernel = GPy.kern.RBF_inv(X.shape[1], ARD=1)

    model = GPy.models.GPRegression(actions, rewards, kernel)

    #model.optimize(optimizer='scg', max_iters = max_iters_opt)

    return model

In [7]:
 model = Create_Gaussian_Model (actions,rewards)
    
print(model)


Name : GP regression
Objective : 1378.9431649700778
Number of Parameters : 64
Number of Optimization Parameters : 64
Updates : True
Parameters:
  GP_regression.           |  value  |  constraints  |  priors
  linear.variances         |  (63,)  |      +ve      |        
  Gaussian_noise.variance  |    1.0  |      +ve      |        


In [8]:
print("model actions")
print(model.X.values)
print(model.X.shape)
print("model rewards")
print(model.Y.values)
print(model.Y.shape)

model actions
[[2.10626984e+00 1.30336192e+00 1.50533547e+00 ... 6.59419705e-04
  1.57288567e-03 7.37282918e-04]
 [6.64042844e-01 1.22560648e+00 1.02339908e+00 ... 1.15477545e-03
  1.39569038e-03 9.31393856e-04]
 [1.19754499e+00 1.80924515e+00 1.39835676e+00 ... 5.52380219e-04
  1.79065083e-03 7.58809772e-04]
 ...
 [1.97407477e+00 6.37037773e-01 1.16377190e+00 ... 1.12702430e-03
  1.26605998e-03 1.19178768e-03]
 [7.96462938e-01 1.29488958e+00 9.74594365e-01 ... 9.36204322e-04
  1.23476868e-03 1.05441314e-03]
 [1.28513665e+00 6.25185039e-01 1.18711875e+00 ... 7.00949285e-04
  2.19622534e-03 1.02368897e-03]]
(1400, 63)
model rewards
[[-0.15615385]
 [-0.17645011]
 [-0.32277424]
 ...
 [-0.21721271]
 [-0.19800912]
 [-0.16752433]]
(1400, 1)


In [9]:
predictions =  model.predict(actions)

error = abs(np.array(predictions[0])-rewards)
error_max = max(error)
error_mean = error.mean(axis=0)

print("error")
print(error)
print("error_max")
print(error_max)
print("error_mean")
print(error_mean)

error
[[0.02802808]
 [0.00159125]
 [0.0224148 ]
 ...
 [0.07392159]
 [0.02252582]
 [0.02936439]]
error_max
[0.26523269]
error_mean
[0.0283789]


In [10]:
# Get the best action from regressed model, by doing random sample

def Get_best_action_from_model_random_sample(env,model,sigma = 0.5, population = 70*10**3,n_elite = 10):
    if(population>70*10**3):
        print("More data fill the memory, and creates and error, used only 70*10**3")
        population = 70*10**3
    
    weights_pop_model = [(sigma*np.random.randn(env.action_space)) for i in range(population)]
    
    actions_model = []
    for weights in weights_pop_model:
        action=np.add(np.multiply(weights,original_action),original_action)
        actions_model.append(action)
    actions_model = np.array(actions_model)
    
    prediction = model.predict(actions_model)
    rewards_model = np.array(prediction[0])
    
    elite_idxs = rewards_model.argsort()[-n_elite:]
    elite_actions = [actions_model[i] for i in elite_idxs]
    elite_rewards = [rewards_model[i] for i in elite_idxs]

    #Set the best weight as the mean of the best ones 

    best_action = np.array(elite_actions).mean(axis=0)
    best_rewards = rewards_model.argsort()[-n_elite:]
    
    return np.array(elite_actions),np.array(elite_rewards)
    
    

In [11]:
# Get the best action from regressed model, by doing random sample

def Get_best_action_from_model_inference(env,model,desired_error = 0):
    for i in range(model.Y.shape[0]):
        desired_error_list.append([desired_error])
    desired_erro_np = np.array(desired_error_list)
    #print(desired_erro_np )
    predicted_actions = model.infer_newX(desired_erro_np , optimize=False)
    predicted_actions = np.array(predicted_actions[0])
    #print(predicted_actions.shape)
    #print(predicted_actions[0,:])
    
    return predicted_actions[0,:]
    
    

In [12]:
"""
best_action,best_rewards = Get_best_action_from_model_random_sample(env,model)
print(best_rewards)
print(best_rewards.shape)
print(best_action)
print(best_action.shape)
"""

#Look which x provide an Y 0
print(rewards)
desired_error_list = []
desired_error = 0
action_infered = Get_best_action_from_model_inference(env,model,desired_error)

[[-0.15615385]
 [-0.17645011]
 [-0.32277424]
 ...
 [-0.21721271]
 [-0.19800912]
 [-0.16752433]]


In [13]:
#Bayesian Search
def Exploration(env,model,best_action_m, pop_size=env.action_space, sigma=0.3):
    
    weights_pop_model = [(sigma*np.random.randn(env.action_space)) for i in range(pop_size)]

    #Test the best action from the model
    actions_explore = [best_action_m]
    rewards_explore = [[env.step_tcp_rishabh(best_action_m)]]
    
    #Test arround it
    for weights in weights_pop_model:
        action=np.add(np.multiply(weights,best_action_m),best_action_m)
        actions_explore.append(action)
        rewards_explore.append([env.step_tcp_rishabh(action)])
        
    #print("rewards explore")
    #print(rewards_explore)

    return actions_explore,rewards_explore
        


In [14]:
Done = False
desired_error = 0
n_iterations = 900
iteration_ch_sigma = 50
sigma = 0.3
sigma_reduction_x_print = 0.65
elite_rewards = []
best_rewards = []
best_reward = -10**8
best_action = np.zeros([1,model.X.shape[1]])


for iteration in range(n_iterations):
    
    #Ask the model the best action and test it and arround it
    best_action_m = Get_best_action_from_model_inference(env,model,desired_error = desired_error)
    actions_explore,rewards_explore = Exploration(env,model,best_action_m, pop_size=env.action_space, sigma=sigma)
    
    rewards_explore = np.array(rewards_explore)
    actions_explore = np.array(actions_explore)
    
    #Sort the result to get the best reward
    elite_idx = rewards_explore.argsort()[-1:]
    if(elite_idx.size >1):
        #print(elite_idx.shape)
        elite_idx = elite_idx[0,0]
        #print(elite_idx.size)
    elite_reward = rewards_explore [elite_idx,0]
    #elite_rewards.append(elite_reward)
    elite_action = actions_explore [elite_idx,:]
    
    
    #print(elite_idx)
    #print(rewards_explore.shape)
        
    if(elite_reward >= best_reward):
        best_reward = elite_reward
        best_rewards.append(best_reward)
        best_action = elite_action
    
    if(elite_reward >= desired_error):
        Done = True
        print("Done")
        break
        
    #Get the data from the actual model
    #actions = model.X.values
    #rewards = model.Y.values
    
    #Add the new data
    #actions = np.vstack((actions,actions_explore))
    #print(rewards)
    #print(rewards_explore)
    #rewards = np.vstack((rewards,rewards_explore))
    
    #Create the model again with this data
    model = Create_Gaussian_Model (actions_explore,rewards_explore)
    
    print("iteration ",iteration)
    print("elite reward",elite_reward)
    print("best reward",best_reward)
    
    
    
    if(iteration%iteration_ch_sigma == 0):
        
        sigma = sigma * sigma_reduction_x_print
    
    
    
    
    
    

iteration  0
elite reward [[-0.12457405]]
best reward [[-0.12457405]]
iteration  1
elite reward [[-0.11566562]]
best reward [[-0.11566562]]
iteration  2
elite reward [[-0.11502929]]
best reward [[-0.11502929]]
iteration  3
elite reward [[-0.09744388]]
best reward [[-0.09744388]]
iteration  4
elite reward [[-0.12349026]]
best reward [[-0.09744388]]
iteration  5
elite reward [[-0.1194274]]
best reward [[-0.09744388]]
iteration  6
elite reward [[-0.11363807]]
best reward [[-0.09744388]]
iteration  7
elite reward [[-0.09317406]]
best reward [[-0.09317406]]
iteration  8
elite reward [[-0.10959857]]
best reward [[-0.09317406]]
iteration  9
elite reward [[-0.08212634]]
best reward [[-0.08212634]]
iteration  10
elite reward [[-0.08012148]]
best reward [[-0.08012148]]
iteration  11
elite reward [[-0.09797402]]
best reward [[-0.08012148]]
iteration  12
elite reward [[-0.11021839]]
best reward [[-0.08012148]]
iteration  13
elite reward [[-0.08987008]]
best reward [[-0.08012148]]
iteration  14
eli

iteration  116
elite reward [[-0.08382293]]
best reward [[-0.06385975]]
iteration  117
elite reward [[-0.07209734]]
best reward [[-0.06385975]]
iteration  118
elite reward [[-0.07544829]]
best reward [[-0.06385975]]
iteration  119
elite reward [[-0.06861644]]
best reward [[-0.06385975]]
iteration  120
elite reward [[-0.07449513]]
best reward [[-0.06385975]]
iteration  121
elite reward [[-0.06945626]]
best reward [[-0.06385975]]
iteration  122
elite reward [[-0.06764242]]
best reward [[-0.06385975]]
iteration  123
elite reward [[-0.06717379]]
best reward [[-0.06385975]]
iteration  124
elite reward [[-0.06764614]]
best reward [[-0.06385975]]
iteration  125
elite reward [[-0.06883284]]
best reward [[-0.06385975]]
iteration  126
elite reward [[-0.06972271]]
best reward [[-0.06385975]]
iteration  127
elite reward [[-0.06732818]]
best reward [[-0.06385975]]
iteration  128
elite reward [[-0.06713836]]
best reward [[-0.06385975]]
iteration  129
elite reward [[-0.06897067]]
best reward [[-0.063

iteration  230
elite reward [[-0.06344816]]
best reward [[-0.06299328]]
iteration  231
elite reward [[-0.06222861]]
best reward [[-0.06222861]]
iteration  232
elite reward [[-0.09888607]]
best reward [[-0.06222861]]
iteration  233
elite reward [[-0.06743307]]
best reward [[-0.06222861]]
iteration  234
elite reward [[-0.0772199]]
best reward [[-0.06222861]]
iteration  235
elite reward [[-0.09610407]]
best reward [[-0.06222861]]
iteration  236
elite reward [[-0.07144064]]
best reward [[-0.06222861]]
iteration  237
elite reward [[-0.14258071]]
best reward [[-0.06222861]]
iteration  238
elite reward [[-0.07233491]]
best reward [[-0.06222861]]
iteration  239
elite reward [[-0.10048437]]
best reward [[-0.06222861]]
iteration  240
elite reward [[-0.06583062]]
best reward [[-0.06222861]]
iteration  241
elite reward [[-0.06625539]]
best reward [[-0.06222861]]
iteration  242
elite reward [[-0.06548648]]
best reward [[-0.06222861]]
iteration  243
elite reward [[-0.06564228]]
best reward [[-0.0622

iteration  345
elite reward [[-0.07545118]]
best reward [[-0.06172313]]
iteration  346
elite reward [[-0.0674127]]
best reward [[-0.06172313]]
iteration  347
elite reward [[-0.06153611]]
best reward [[-0.06153611]]
iteration  348
elite reward [[-0.09261311]]
best reward [[-0.06153611]]
iteration  349
elite reward [[-0.06232361]]
best reward [[-0.06153611]]
iteration  350
elite reward [[-0.0917125]]
best reward [[-0.06153611]]
iteration  351
elite reward [[-0.06315532]]
best reward [[-0.06153611]]
iteration  352
elite reward [[-0.06679123]]
best reward [[-0.06153611]]
iteration  353
elite reward [[-0.06270072]]
best reward [[-0.06153611]]
iteration  354
elite reward [[-0.06197725]]
best reward [[-0.06153611]]
iteration  355
elite reward [[-0.0631222]]
best reward [[-0.06153611]]
iteration  356
elite reward [[-0.06160629]]
best reward [[-0.06153611]]
iteration  357
elite reward [[-0.06319025]]
best reward [[-0.06153611]]
iteration  358
elite reward [[-0.06212759]]
best reward [[-0.061536

iteration  460
elite reward [[-0.06153642]]
best reward [[-0.06054686]]
iteration  461
elite reward [[-0.06249394]]
best reward [[-0.06054686]]
iteration  462
elite reward [[-0.08192768]]
best reward [[-0.06054686]]
iteration  463
elite reward [[-0.06215493]]
best reward [[-0.06054686]]
iteration  464
elite reward [[-0.06319531]]
best reward [[-0.06054686]]
iteration  465
elite reward [[-0.06144798]]
best reward [[-0.06054686]]
iteration  466
elite reward [[-0.06084167]]
best reward [[-0.06054686]]
iteration  467
elite reward [[-0.06142755]]
best reward [[-0.06054686]]
iteration  468
elite reward [[-0.06124462]]
best reward [[-0.06054686]]
iteration  469
elite reward [[-0.06143374]]
best reward [[-0.06054686]]
iteration  470
elite reward [[-0.06149678]]
best reward [[-0.06054686]]
iteration  471
elite reward [[-0.06128863]]
best reward [[-0.06054686]]
iteration  472
elite reward [[-0.06214464]]
best reward [[-0.06054686]]
iteration  473
elite reward [[-0.06236441]]
best reward [[-0.060

iteration  575
elite reward [[-0.06340141]]
best reward [[-0.06021253]]
iteration  576
elite reward [[-0.06281437]]
best reward [[-0.06021253]]
iteration  577
elite reward [[-0.06188194]]
best reward [[-0.06021253]]
iteration  578
elite reward [[-0.06211212]]
best reward [[-0.06021253]]
iteration  579
elite reward [[-0.06237318]]
best reward [[-0.06021253]]
iteration  580
elite reward [[-0.06246476]]
best reward [[-0.06021253]]
iteration  581
elite reward [[-0.06196138]]
best reward [[-0.06021253]]
iteration  582
elite reward [[-0.06238866]]
best reward [[-0.06021253]]
iteration  583
elite reward [[-0.06194786]]
best reward [[-0.06021253]]
iteration  584
elite reward [[-0.0621864]]
best reward [[-0.06021253]]
iteration  585
elite reward [[-0.06236915]]
best reward [[-0.06021253]]
iteration  586
elite reward [[-0.06215919]]
best reward [[-0.06021253]]
iteration  587
elite reward [[-0.06144904]]
best reward [[-0.06021253]]
iteration  588
elite reward [[-0.06349936]]
best reward [[-0.0602

iteration  689
elite reward [[-0.06168729]]
best reward [[-0.06021253]]
iteration  690
elite reward [[-0.06096334]]
best reward [[-0.06021253]]
iteration  691
elite reward [[-0.06190122]]
best reward [[-0.06021253]]
iteration  692
elite reward [[-0.06152659]]
best reward [[-0.06021253]]
iteration  693
elite reward [[-0.06229142]]
best reward [[-0.06021253]]
iteration  694
elite reward [[-0.06217295]]
best reward [[-0.06021253]]
iteration  695
elite reward [[-0.06208146]]
best reward [[-0.06021253]]
iteration  696
elite reward [[-0.06168121]]
best reward [[-0.06021253]]
iteration  697
elite reward [[-0.06126505]]
best reward [[-0.06021253]]
iteration  698
elite reward [[-0.06135956]]
best reward [[-0.06021253]]
iteration  699
elite reward [[-0.06152557]]
best reward [[-0.06021253]]
iteration  700
elite reward [[-0.0614059]]
best reward [[-0.06021253]]
iteration  701
elite reward [[-0.06178554]]
best reward [[-0.06021253]]
iteration  702
elite reward [[-0.06188829]]
best reward [[-0.0602

iteration  803
elite reward [[-0.0611763]]
best reward [[-0.06021253]]
iteration  804
elite reward [[-0.06140578]]
best reward [[-0.06021253]]
iteration  805
elite reward [[-0.06141539]]
best reward [[-0.06021253]]
iteration  806
elite reward [[-0.06134905]]
best reward [[-0.06021253]]
iteration  807
elite reward [[-0.06139825]]
best reward [[-0.06021253]]
iteration  808
elite reward [[-0.06147218]]
best reward [[-0.06021253]]
iteration  809
elite reward [[-0.06146489]]
best reward [[-0.06021253]]
iteration  810
elite reward [[-0.0614253]]
best reward [[-0.06021253]]
iteration  811
elite reward [[-0.06140575]]
best reward [[-0.06021253]]
iteration  812
elite reward [[-0.06121586]]
best reward [[-0.06021253]]
iteration  813
elite reward [[-0.06144618]]
best reward [[-0.06021253]]
iteration  814
elite reward [[-0.06118919]]
best reward [[-0.06021253]]
iteration  815
elite reward [[-0.06143682]]
best reward [[-0.06021253]]
iteration  816
elite reward [[-0.06144653]]
best reward [[-0.06021

In [15]:
model.objective_function()

66.76182781286387

In [16]:
print("iteration ",iteration)
print("elite reward",elite_reward)
print("best reward",best_reward)

iteration  899
elite reward [[-0.06130904]]
best reward [[-0.06021253]]


In [17]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores,best_actions = cem_no_net()
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
    

NameError: name 'cem_no_net' is not defined

In [ ]:

# plot the last scores zoom
fig = plt.figure()
zoom= 300
ax = fig.add_subplot(111)
plt.plot(np.arange(1, zoom+1), scores[-zoom:])
plt.ylabel('Score zoom')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# plot actions
best_actions_np = np.array(best_actions)
joint = 1
for i in range(len(env.parameters_to_modify)) :
    parameter = env.parameters_to_modify[i]
    figures = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, best_actions_np.shape[0]+1), best_actions_np[:,joint+i*7])
    plt.ylabel(parameter+" Joint"+str(joint))
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz","damping"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.create_df_from_Excel("./Parameters_train_tcp_euc_rishabh.xlsx")


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous
print("trained",action)
reward = env.step_tcp_rishabh(action)
print("reward")
print(reward)



In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Train_parameters_result_tcp_euc_rishabh.xlsx")

In [ ]:
env.original_parameters_df

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.original_parameters_df


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print(action)
reward = env.step_tcp_rishabh(action)

print("reward")
print(reward)

In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Original_parameters_result_tcp_euc_rishabh.xlsx")